# Train Music Transformer
Since Fri. May 13th, 2022

Set up training in colab


## Setup


### Ipython



In [3]:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Colab



In [4]:
import os
import sys

import torch


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')

    ! pip3 install sty icecream music21 transformers datasets

    path = os.path.join('drive', 'My Drive', 'Research', 'Music with NLP', 'Symbolic-Music-Generation')
    sys.path.append(path)
    ! ls "{path}"


    import time, os
    os.environ['TZ'] = 'US/Eastern'
    time.tzset()

    if torch.cuda.is_available():
        %env PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:128

if torch.cuda.is_available():
    ! nvidia-smi


from icecream import ic

from musicnlp.util import *

ic(u.proj_dir, u.pkg_nm)



Mounted at /content/drive
     |████████████████████████████████| 4.0 MB 5.5 MB/s 
     |████████████████████████████████| 325 kB 73.4 MB/s 
     |████████████████████████████████| 895 kB 56.6 MB/s 
     |████████████████████████████████| 6.6 MB 55.5 MB/s 
     |████████████████████████████████| 77 kB 9.0 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 136 kB 77.3 MB/s 
     |████████████████████████████████| 1.1 MB 41.0 MB/s 
     |████████████████████████████████| 212 kB 75.0 MB/s 
     |████████████████████████████████| 127 kB 73.5 MB/s 
     |████████████████████████████████| 271 kB 73.0 MB/s 
     |████████████████████████████████| 144 kB 72.2 MB/s 
     |████████████████████████████████| 94 kB 4.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: Py

ic| PATH_BASE: 'drive/My Drive/Research/Music with NLP'
    DIR_PROJ: 'Symbolic-Music-Generation'
    PKG_NM: 'musicnlp'


('drive/My Drive/Research/Music with NLP',
 'Symbolic-Music-Generation',
 'musicnlp')

### code

In [5]:
import transformers

from musicnlp.trainer import train

transformers.set_seed(sconfig('random-seed'))



## Prep for training


In [6]:
md_nm = 'transf-xl'
# md_sz = 'debug'
# md_sz = 'tiny'
md_sz = 'base'


augment_key = False
dnm_909 = 'musicnlp music extraction, dnm=POP909, n=909, ' \
          'meta={mode=melody, prec=5, th=1}, 2022-04-16_20-28-47'
dnm_lmd = 'musicnlp music extraction, dnm=LMD-cleaned-subset, n=10269, ' \
          'meta={mode=melody, prec=5, th=1}, 2022-04-17_11-52-15'
# dnms = [dnm_909, dnm_lmd]
dnms = [dnm_909]


if md_sz == 'debug':
    n = 32
    train_args = dict(
        per_device_train_batch_size=4,
        # save_strategy='no',
        save_strategy='epoch',
        num_train_epochs=64,
    )
    my_train_args = dict(
        logging_strategy='epoch',
        save_epochs=16,
        tqdm='train-only',
        augment_key=augment_key
    )
elif md_sz == 'tiny':
    n = 64
    train_args = dict(
        per_device_train_batch_size=32,
        save_strategy='epoch',
        num_train_epochs=64,
    )
    my_train_args = dict(
        save_epochs=16,
        tqdm='train-only',
        augment_key=augment_key
)
else:
    n = None
    train_args = dict(
        num_train_epochs=64,
        per_device_train_batch_size=32,
    )
    my_train_args = dict(
        fp16=torch.cuda.is_available(),
        logging_strategy='epoch',
        save_epochs=2,
        tqdm='train-only',
        augment_key=augment_key
    )

mdl, tokenizer, trainer = train.get_all_setup(
    model_name=md_nm, model_size=md_sz, dataset_names=dnms, n_sample=n,
    train_args=train_args, my_train_args=my_train_args
)



## Train


Check log and tensorboard files written



In [ ]:
# trainer.train()
checkpoint_path = os.path.join(PATH_BASE, DIR_PROJ, DIR_MDL, 'reformer', '2022-04-17_22-53-41', 'checkpoint-3430')
trainer.train(checkpoint_path)

trainer.save_model(os.path.join(trainer.args.output_dir, 'trained'))


2022-04-18 08:56:30|[ReformerModelWithLMHead Training]::on_train_begin::train.py:275, INFO - Training started with model {model name: ReformerModelWithLMHead, max length: 2048, axial_pos_shape: (32, 64), n_layer: 12, hidden_size: 768, ff_size: 3072, attention_shape: 12x64, parameter_count: 39.9M}, {
    "hash_seed": null,
    "vocab_size": 462,
    "attention_head_size": 64,
    "hidden_size": 768,
    "num_attention_heads": 12,
    "num_hashes": 2,
    "num_hidden_layers": 6,
    "num_buckets": null,
    "lsh_attn_chunk_length": 64,
    "local_attn_chunk_length": 64,
    "lsh_num_chunks_after": 0,
    "lsh_num_chunks_before": 1,
    "local_num_chunks_after": 0,
    "local_num_chunks_before": 1,
    "hidden_act": "relu",
    "feed_forward_size": 3072,
    "hidden_dropout_prob": 0.05,
    "lsh_attention_probs_dropout_prob": 0.0,
    "local_attention_probs_dropout_prob": 0.05,
    "max_position_embeddings": 2048,
    "initializer_range": 0.02,
    "layer_norm_eps": 1e-12,
    "axial_pos_

Epoch 11/64:   0%|          | 0/343 [00:00<?, ?it/s]config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...
config.num_buckets is not set. Setting config.num_buckets to 64...
Epoch 11/64: 100%|██████████| 343/343 [38:50<00:00,  5.32s/it]

2022-04-18 09:35:32|[ReformerModelWithLMHead Training]::_log::train.py:343, INFO - {step:  3773/21952, epoch: 11.000/64, train/learning_rate: 2.953e-04, train/loss: 1.5178, train/ntp_acc: 55.19, train/ikr: 69.73}


Epoch 11/64: 100%|██████████| 343/343 [38:50<00:00,  6.79s/it]


2022-04-18 09:35:58|[ReformerModelWithLMHead Training]::_log::train.py:343, INFO - {step:  3773/21952, epoch: 11/64, eval/loss: 1.3271, eval/ntp_acc: 60.95, eval/ikr: 82.95}


...

Epoch 18/64: 100%|██████████| 343/343 [40:20<00:00,  5.53s/it]

2022-04-18 14:20:43|[ReformerModelWithLMHead Training]::_log::train.py:343, INFO - {step:  6174/21952, epoch: 18.000/64, train/learning_rate: 2.710e-04, train/loss: 1.3089, train/ntp_acc: 60.67, train/ikr: 86.83}


Epoch 18/64: 100%|██████████| 343/343 [40:23<00:00,  7.07s/it]


2022-04-18 14:21:13|[ReformerModelWithLMHead Training]::_log::train.py:343, INFO - {step:  6174/21952, epoch: 18/64, eval/loss: 1.1163, eval/ntp_acc: 67.43, eval/ikr: 82.54}


Epoch 19/64:  36%|███▌      | 123/343 [14:28<25:52,  7.06s/it]

In [ ]:
ic(trainer.args.output_dir)
ic(os.listdir(trainer.args.output_dir))

